In [28]:
import logging
import sys
from IPython.display import Markdown, display

In [29]:
print(f"Installing packages to {sys.executable}")

Installing packages to c:\anaconda3\envs\pydurable\python.exe


In [30]:
!{sys.executable} -m pip install snowflake-connector-python[pandas]
#!{sys.executable} -m pip install azure-identity
#!{sys.executable} -m pip install azure-storage-blob
#!{sys.executable} -m pip install python-dotenv

In [31]:
import snowflake.connector
import time
import os

# Setup Snowflake Connection

In [32]:
USER = ""
PASSWORD = ""
ACCOUNT = ""
WAREHOUSE = ""
DATABASE = ""
SCHEMA = ""

In [33]:
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=SCHEMA
)

# SQL query to Run

In [34]:
sql_query = f"SELECT * FROM LINEITEM WHERE L_ORDERKEY >= 0 AND L_ORDERKEY <= 40000000"

In [35]:
import uuid

staging_folder = str(uuid.uuid4().hex) 
staging_folder

'e7cf6d1dfd04487581dc1ecfe9d01d93'

In [36]:
sub_query = "SELECT * FROM LINEITEM WHERE L_ORDERKEY >= 0 AND L_ORDERKEY <= 40000000"
staging_query_lines = [f"copy into @~/{staging_folder}",
    f"from ({sql_query})",
    "FILE_FORMAT=(TYPE=parquet, COMPRESSION=SNAPPY)",
    "MAX_FILE_SIZE = 268435456",
    "INCLUDE_QUERY_ID=true;"
]

staging_query = " ".join(staging_query_lines)
staging_query

'copy into @~/e7cf6d1dfd04487581dc1ecfe9d01d93 from (SELECT * FROM LINEITEM WHERE L_ORDERKEY >= 0 AND L_ORDERKEY <= 40000000) FILE_FORMAT=(TYPE=parquet, COMPRESSION=SNAPPY) MAX_FILE_SIZE = 268435456 INCLUDE_QUERY_ID=true;'

# Create Download Folder

In [37]:
base_folder = f"."

local_folder = f"{base_folder}/{staging_folder}"

In [38]:
import os 

if not os.path.exists(local_folder):
    os.mkdir(local_folder)
    print(f"Created local folder for downloaded files {local_folder}")
else:
    print(f"Local folder exists for downloaded files {local_folder} (absolute path: {os.path.abspath(local_folder)})")

# Query to Download Files

In [39]:
get_query = f"GET @~/{staging_folder} file://{local_folder};"
get_query

'GET @~/e7cf6d1dfd04487581dc1ecfe9d01d93 file://./e7cf6d1dfd04487581dc1ecfe9d01d93;'

# Query to Remove Staged Files

In [40]:
rm_query = f"RM @~/{staging_folder}"
rm_query

'RM @~/e7cf6d1dfd04487581dc1ecfe9d01d93'

# Run Queries against Database

In [41]:
%%time
print(f"Running staging query: {staging_query}")
conn.cursor().execute(staging_query)

CPU times: total: 0 ns
Wall time: 40 s


In [42]:
%%time
print(f"Running query to download files: {get_query}")
conn.cursor().execute(get_query)

Running query to download files: GET @~/e7cf6d1dfd04487581dc1ecfe9d01d93 file://./e7cf6d1dfd04487581dc1ecfe9d01d93;
CPU times: total: 1min 17s
Wall time: 7min 22s


In [45]:
print(f"Files downloaded to {os.path.abspath(local_folder)}:")

for f in os.listdir(local_folder):
    file_path = os.path.join(local_folder, f)

    if os.path.isfile(file_path):
        os.path.getsize(file_path)
        file_size_mb = os.path.getsize(file_path)/(1024*1024)
        print(f"{f}: {file_size_mb} MB")

Files downloaded to c:\source_code\github.com\rivms\dbqueryfunc\e7cf6d1dfd04487581dc1ecfe9d01d93:
data_01aeec3f-0001-5699-0004-725600015032_0_0_0.snappy.parquet: 203.49921989440918 MB
data_01aeec3f-0001-5699-0004-725600015032_0_1_0.snappy.parquet: 202.16798973083496 MB
data_01aeec3f-0001-5699-0004-725600015032_0_2_0.snappy.parquet: 201.43767833709717 MB
data_01aeec3f-0001-5699-0004-725600015032_0_3_0.snappy.parquet: 213.98997497558594 MB
data_01aeec3f-0001-5699-0004-725600015032_0_4_0.snappy.parquet: 199.45056819915771 MB
data_01aeec3f-0001-5699-0004-725600015032_0_5_0.snappy.parquet: 194.73483848571777 MB
data_01aeec3f-0001-5699-0004-725600015032_0_6_0.snappy.parquet: 206.14772701263428 MB
data_01aeec3f-0001-5699-0004-725600015032_0_7_0.snappy.parquet: 202.4014482498169 MB


In [43]:
%%time
print(f"Running query to remove staging files: {get_query}")

conn.cursor().execute(rm_query)

Running query to remove staging files: GET @~/e7cf6d1dfd04487581dc1ecfe9d01d93 file://./e7cf6d1dfd04487581dc1ecfe9d01d93;
CPU times: total: 844 ms
Wall time: 3.34 s


In [44]:
# Close the cursor and the connection.
conn.close()